## Para fazer:

converter pandas dataframes to tensors
https://stackoverflow.com/questions/65767833/pandas-dataframe-to-tensor

perceber cada linha de codigo ao mais infimo detalhe

perceber mesmo o que é um tensor bem

tiar collum do cancro do pulmão. fai servir para delta e weight delta ou para aume tat o erro na função de sum of scare residuals

tornar todos os nomes com letras pequenas e não existir espaos nos nomes

ver codigo exemplo em: https://github.com/IEEE-NOVA-SB/Easy_PyTorch_Models

In [1]:
import torch #create tensors to raw data and weights
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F #gives activation function  
from torch.optim import SGD #stochastic gradient descend
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r'data.csv')
pd.options.display.max_columns = None

In [3]:
df['LUNG_CANCER'] = df['LUNG_CANCER'].map({'YES': 1, 'NO': 0})
df['GENDER'] = df['GENDER'].map({'M': 1, 'F': 0})
collumsToChange = ["SMOKING", "YELLOW_FINGERS","ANXIETY","PEER_PRESSURE","CHRONIC DISEASE","FATIGUE","ALLERGY","WHEEZING","ALCOHOL CONSUMING","COUGHING","SHORTNESS OF BREATH","SWALLOWING DIFFICULTY","CHEST PAIN"] 
for collum in collumsToChange:
    df[collum] = df[collum].map({2: 1, 1: 0})

print(df.head(5))

   GENDER  AGE  SMOKING  YELLOW_FINGERS  ANXIETY  PEER_PRESSURE  \
0       1   69        0               1        1              0   
1       1   74        1               0        0              0   
2       0   59        0               0        0              1   
3       1   63        1               1        1              0   
4       0   63        0               1        0              0   

   CHRONIC DISEASE  FATIGUE  ALLERGY  WHEEZING  ALCOHOL CONSUMING  COUGHING  \
0                0        1        0         1                  1         1   
1                1        1        1         0                  0         0   
2                0        1        0         1                  0         1   
3                0        0        0         0                  1         0   
4                0        0        0         1                  0         1   

   SHORTNESS OF BREATH  SWALLOWING DIFFICULTY  CHEST PAIN  LUNG_CANCER  
0                    1                      1    

## Porque considerei que os homens é 1 e mulheres é 0?

Porque os homens têm um metabolismo mais alto, mair chance de trabalhar em ambientes poluidos(cavernas), entre outros.

Assim, na parte do género tenta-se compensar estas variaveis não quantificaveis

In [ ]:
train = df.sample(frac=0.8,random_state=0)
test = df.drop(train.index)

train=torch.tensor(x,dtype=torch.float32)
test=torch.tensor(y,dtype=torch.float32)

In [ ]:
batch_size_num = 64

train_dataloader = DataLoader(train, batch_size=batch_size_num, shuffle=True)
test_dataloader = DataLoader(test, batch_size=batch_size_num, shuffle=True)

In [ ]:
print(train_dataloader)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(16, 10),
            nn.ReLU(),
            nn.Linear(10, 5),
            nn.ReLU(),
            nn.Linear(5, 1)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)



In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")